# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import shap
import pickle

columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))


# Get X and Y

Create the features data frame and target data.

In [15]:

# Create feature dataframe X and target dataframe Y
X = fires_dt.drop(columns=['area'], axis=1)
y = fires_dt['area']

# Convert categorical columns
X = pd.get_dummies(X, columns=['month', 'day'])

# Fill missing values
X.fillna(0, inplace=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [16]:

preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [17]:
# Preproc 2 (with MinMaxScaler as an example of non-linear transformation)
preproc2 = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [18]:
# Baseline regressor: Linear Regression
baseline = LinearRegression()

# Advanced regressor: Random Forest
advanced = RandomForestRegressor(random_state=42)

# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline(steps=[('preprocessing', preproc1), ('regressor', baseline)])


In [19]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(steps=[('preprocessing', preproc2), ('regressor', baseline)])

In [20]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(steps=[('preprocessing', preproc1), ('regressor', advanced)])



In [21]:
pipeline_D = Pipeline(steps=[('preprocessing', preproc2), ('regressor', advanced)])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [22]:
param_grid_A = {'regressor__fit_intercept': [True, False]}
param_grid_B = {'regressor__fit_intercept': [True, False]}
param_grid_C = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_features': ['auto', 'sqrt', 'log2']}
param_grid_D = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_features': ['auto', 'sqrt', 'log2']}

grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5)
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5)
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5)
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5)

In [ ]:
# Fit and evaluate
grid_A.fit(X_train, y_train)
grid_B.fit(X_train, y_train)
grid_C.fit(X_train, y_train)
grid_D.fit(X_train, y_train)

# Evaluate each model
results = {
    'pipeline_A': mean_squared_error(y_test, grid_A.predict(X_test)),
    'pipeline_B': mean_squared_error(y_test, grid_B.predict(X_test)),
    'pipeline_C': mean_squared_error(y_test, grid_C.predict(X_test)),
    'pipeline_D': mean_squared_error(y_test, grid_D.predict(X_test)),
}

best_pipeline = min(results, key=results.get)
print(f"The best performing model is {best_pipeline} with MSE: {results[best_pipeline]}")

In [ ]:
# Save the best performing model to a pickle file
best_model = {
    'pipeline_A': grid_A,
    'pipeline_B': grid_B,
    'pipeline_C': grid_C,
    'pipeline_D': grid_D,
}[best_pipeline]

with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [ ]:
# Explainability with SHAP
explainer = shap.Explainer(best_model.best_estimator_.named_steps['regressor'], X_train)
shap_values = explainer(X_test)

# Select an observation and explain
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[0])

# General feature importance
shap.summary_plot(shap_values, X_test)

# Determine features to remove
feature_importance = shap_values.abs.mean(0).values
features_to_remove = [X_train.columns[i] for i in range(len(feature_importance)) if feature_importance[i] < 0.01]
print(f"Features to consider removing: {features_to_remove}")

# Test the impact of removing features
X_train_reduced = X_train.drop(features_to_remove, axis=1)
X_test_reduced = X_test.drop(features_to_remove, axis=1)

best_model.fit(X_train_reduced, y_train)
reduced_performance = mean_squared_error(y_test, best_model.predict(X_test_reduced))
print(f"MSE with reduced features: {reduced_performance}")

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.